In [3]:
! pip install PyMySQL

     |████████████████████████████████| 43 kB 1.8 MB/s 


In [4]:
import pymysql.cursors
import pandas as pd
# 데이터베이스 연결
connection = pymysql.connect(host = '13.124.77.49',
                             user='USER_SQLSTK',
                             password='1qaz2wsx',
                             database='DB_SQLSTK',
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor                             
                             )

cursor = connection.cursor()

# 실행할 SQL
# 2019년 1월 1일 ~ 2020년 12월 31일 셀트리온 종가, 시가, 고가, 저가, 거래량, (5, 10, 20) 이평선
sql = """
      SELECT T1.STK_CD, T1.STK_NM, T1.SEC_NM
            ,T2.DT, O_PRC, T2.L_PRC, T2.H_PRC, T2.C_PRC, T2.VOL
            ,M5_PRC, M10_PRC, M20_PRC
      FROM stock T1
      INNER JOIN HISTORY_DT T2
      ON (T2.STK_CD = T1.STK_CD)
      WHERE T1.STK_NM = '셀트리온'
      AND T2.DT >= '2019-01-01'
      AND T2.DT <= '2019-03-31';
  """

# SQL 쿼리를 실행할 준비
cursor.execute(sql)

# SQL 쿼리를 실행하고 결과 result
result = cursor.fetchall()

# sql을 데이터프레임으로 
df = pd.DataFrame(result)
df.head()

,STK_CD,STK_NM,SEC_NM,DT,O_PRC,L_PRC,H_PRC,C_PRC,VOL,M5_PRC,M10_PRC,M20_PRC
0,068270,셀트리온,제약바이오,2019-01-02,210559.000,199773.000,211965.000,201180.000,1114572.000,204618.600,203904.700,209007.600
1,068270,셀트리온,제약바이오,2019-01-03,200711.000,195552.000,203993.000,197428.000,1014921.000,205309.000,203881.300,207797.950
2,068270,셀트리온,제약바이오,2019-01-04,197428.000,195083.000,207276.000,206338.000,1207224.000,204556.200,204380.300,206895.600
3,068270,셀트리온,제약바이오,2019-01-07,212903.000,201649.000,212903.000,202586.000,1142715.000,203243.000,203859.100,205874.750
4,068270,셀트리온,제약바이오,2019-01-08,203055.000,199304.000,205400.000,200242.000,689714.000,201554.800,203241.700,205036.200


In [5]:
# from datetime import datetime
df1 = pd.read_csv("celltrion_m60_prc.csv", engine="python") # 셀트리온 60일 이평선이 들어간 csv
df1['60days'] = df1['c_prc']/df1['M60_PRC'] * 100 # 60일 이평선의 이격도
df['M60_PRC'] = df1['M60_PRC'] # 기존 df에 m60_prc 라는 열을 만들고 그 안에 df1에서 가져온 60일 이평선 대입
df['20days'] = df['C_PRC']/df['M20_PRC']*100 # 20일 이평선의 이격도
df['60days'] = df1['60days'] # 기존 df에 60일 이격도 대입
df1.head()

,dt,stk_nm,c_prc,M60_PRC,60days
0,2019-01-02,셀트리온,201180.0,217355.000,92.558257
1,2019-01-03,셀트리온,197428.0,216206.900,91.314385
2,2019-01-04,셀트리온,206338.0,215191.950,95.885557
3,2019-01-07,셀트리온,202586.0,214168.217,94.592000
4,2019-01-08,셀트리온,200242.0,213335.800,93.862352


In [6]:
df.head()

,STK_CD,STK_NM,SEC_NM,DT,O_PRC,L_PRC,H_PRC,C_PRC,VOL,M5_PRC,M10_PRC,M20_PRC,M60_PRC,20days,60days
0,068270,셀트리온,제약바이오,2019-01-02,210559.000,199773.000,211965.000,201180.000,1114572.000,204618.600,203904.700,209007.600,217355.000,96.25487302854058895466002193,92.558257
1,068270,셀트리온,제약바이오,2019-01-03,200711.000,195552.000,203993.000,197428.000,1014921.000,205309.000,203881.300,207797.950,216206.900,95.00959946909967109877647975,91.314385
2,068270,셀트리온,제약바이오,2019-01-04,197428.000,195083.000,207276.000,206338.000,1207224.000,204556.200,204380.300,206895.600,215191.950,99.73049209359696387936717842,95.885557
3,068270,셀트리온,제약바이오,2019-01-07,212903.000,201649.000,212903.000,202586.000,1142715.000,203243.000,203859.100,205874.750,214168.217,98.40254815124244231019102634,94.592000
4,068270,셀트리온,제약바이오,2019-01-08,203055.000,199304.000,205400.000,200242.000,689714.000,201554.800,203241.700,205036.200,213335.800,97.66177874931353585366876678,93.862352


In [7]:
# 그래프에 넣을 때 오류를 방지하기 위해 date 타입으로 변경
df["DT"]=pd.to_datetime(df["DT"])
# O_PRC 정수로 변경
df.loc[: , "O_PRC":] = df.loc[: , "O_PRC":].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   STK_CD   59 non-null     object        
 1   STK_NM   59 non-null     object        
 2   SEC_NM   59 non-null     object        
 3   DT       59 non-null     datetime64[ns]
 4   O_PRC    59 non-null     int64         
 5   L_PRC    59 non-null     int64         
 6   H_PRC    59 non-null     int64         
 7   C_PRC    59 non-null     int64         
 8   VOL      59 non-null     int64         
 9   M5_PRC   59 non-null     int64         
 10  M10_PRC  59 non-null     int64         
 11  M20_PRC  59 non-null     int64         
 12  M60_PRC  59 non-null     int64         
 13  20days   59 non-null     int64         
 14  60days   59 non-null     int64         
dtypes: datetime64[ns](1), int64(11), object(3)
memory usage: 7.0+ KB


In [8]:
! pip install plotly==5.2.1

     |████████████████████████████████| 21.8 MB 67.9 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [9]:
import plotly.graph_objects as go

# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

ch = go.Figure(data=candle) # 봉차트 candle을 그릴 준비를 함
ch.show() # 봉차트 출력

In [10]:
# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.DT, y=df['VOL'])

In [11]:
import plotly.subplots as ms
# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)

# row=1, col=1 : 1번째줄 1번째 칸에 "Stock Price" 제목 출력
fig.update_yaxes(title_text="Stock Price", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "Volume" 제목 출력
fig.update_yaxes(title_text="Volume", row=2, col=1)
# 셀트리온 제목 출력
fig.update_layout( title='셀트리온' )

In [12]:
# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.DT, y=df['VOL'])

# 5일 이동 평균 선 차트 
ma5 = go.Scatter(x=df.DT, y=df['M5_PRC'], line=dict(color='black', width=0.8), name='ma5')
# 10일 이동 평균 선 차트 
ma10 = go.Scatter(x=df.DT, y=df['M10_PRC'], line=dict(color='red', width=0.9), name='ma10')
# 20일 이동 평균 선 차트 
ma20 = go.Scatter(x=df.DT, y=df['M20_PRC'], line=dict(color='green', width=1), name='ma20')
# 60일 이동 평균 선 차트
ma60 = go.Scatter(x=df.DT, y=df['M60_PRC'], line=dict(color='purple', width=2), name='ma60')
# 20일 이격도 차트
day20 = go.Scatter(x=df.DT, y=df['20days'], line=dict(color='green', width=2), name='day20')
# 60일 이격도 차트
day60 = go.Scatter(x=df.DT, y=df['60days'], line=dict(color='purple', width=2), name='day60')

# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma5 차트 출력
fig.add_trace(ma5, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma10 차트 출력
fig.add_trace(ma10, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma20 차트 출력
fig.add_trace(ma20, row=1, col=1)
# orw=1, col=1 : 1번째 줄 1번째 칸에 ma60 차트 출력
fig.add_trace(ma60, row=1, col=1)

# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 20days 차트 출력
fig.add_trace(day20, row=3, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 60days 차트 출력
fig.add_trace(day60, row=3, col=1)
# 이평선 90 이하일 때 매수
fig.add_hline(y=90, line=dict(color='black', width=2), name='cutline90') 
# 이평선 110 이상일 때 매수
fig.add_hline(y=110, line=dict(color='red', width=2), name='cutline110') 

# row=1, col=1 : 1번째줄 1번째 칸에 "주가" 제목 출력
fig.update_yaxes(title_text="주가", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "거래량" 제목 출력
fig.update_yaxes(title_text="거래량", row=2, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 "이격도" 제목 출력
fig.update_yaxes(title_text="이격도", row=3, col=1)

# 셀트리온 제목 출력
fig.update_layout( title='셀트리온')

#최종

In [13]:
# 이격도 차트
fig = ms.make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)
# row=3, col=1 : 3번째줄 1번째 칸에 20days 차트 출력
fig.add_trace(day20, row=1, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 60days 차트 출력
fig.add_trace(day60, row=1, col=1)
# 이평선 90 이하일 때 매수
fig.add_hline(y=90, line=dict(color='black', width=2), name='cutline90') 
# 이평선 110 이상일 때 매수
fig.add_hline(y=110, line=dict(color='red', width=2), name='cutline110') 

In [14]:
# 봉차트 + 거래량
# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)

# row=1, col=1 : 1번째줄 1번째 칸에 "Stock Price" 제목 출력
fig.update_yaxes(title_text="Stock Price", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "Volume" 제목 출력
fig.update_yaxes(title_text="Volume", row=2, col=1)
# 셀트리온 제목 출력
fig.update_layout( title='셀트리온' )